### Install libraries

In [1]:
!pip install pandas matplotlib seaborn scikit-learn
!pip install kaggle

In [2]:
!pip install python-dotenv

### Import libraries

In [28]:
import pandas as pd
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
import os
from dotenv import load_dotenv

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading dataset from Kaggle

In [6]:
!pwd

/content


In [7]:
!ls "/content/drive/My Drive/Colab Notebooks/Datasets"

vehicles_dataset.csv


In [24]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/vehicles_dataset.csv")

### Loading vars and cloning Github Repo

In [9]:
!pwd

/content


In [10]:
!ls /content/

ci-cd-project  drive  sample_data


In [11]:
# remove old version of git repo if it still exists from last time
!rm -rf /content/ci-cd-project

In [12]:
# clone the current repo
!git clone https://github.com/empt22/ci-cd-project.git

Cloning into 'ci-cd-project'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 68 (delta 16), reused 57 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 12.04 KiB | 205.00 KiB/s, done.
Resolving deltas: 100% (16/16), done.


In [13]:
%cd /content/ci-cd-project

/content/ci-cd-project


In [14]:
# load environment variables from .env file
load_dotenv("/content/drive/My Drive/Colab Notebooks/config/.env")
token = os.getenv("GITHUB_PAT")
user_name = os.getenv("GIT_USER_NAME")
user_email = os.getenv("GIT_USER_EMAIL")

# Use the token for Git commands
!git remote set-url origin https://{token}@github.com/empt22/ci-cd-project.git

In [15]:
# confirm user identify who is making commits to Github
!git config --global user.name "{user_name}"
!git config --global user.email "{user_email}"

### Data analysis

In [29]:
len(df)

1002

In [30]:
df.head()

,name,description,make,model,type,year,price,engine,cylinders,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,New,2024,74600.0,24V GDI DOHC Twin Turbo,6.0,Gasoline,10.0,8-Speed Automatic,Series II,SUV,4.0,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,New,2024,50170.0,OHV,6.0,Gasoline,1.0,8-Speed Automatic,Laredo,SUV,4.0,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,New,2024,96410.0,"6.2L V-8 gasoline direct injection, variable v...",8.0,Gasoline,0.0,Automatic,Denali,SUV,4.0,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,New,2023,46835.0,16V MPFI OHV,8.0,Gasoline,32.0,8-Speed Automatic,Pursuit,SUV,4.0,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,New,2024,81663.0,24V DDI OHV Turbo Diesel,6.0,Diesel,10.0,6-Speed Automatic,Laramie,Pickup Truck,4.0,Silver,Black,Four-wheel Drive


#### Summary stats

In [39]:
df['make'].value_counts()

,count
make,
Jeep,194
Hyundai,121
Dodge,117
Ford,88
RAM,79
Kia,52
Chevrolet,49
Nissan,40
Volkswagen,38


In [40]:
df['year'].value_counts()

,count
year,
2024,906
2023,90
2025,6


In [41]:
df['mileage'].value_counts()

,count
mileage,
5.0,116
0.0,110
10.0,108
1.0,58
6.0,50
...,...
447.0,1
73.0,1
399.0,1


In [48]:
print(df.isnull().sum())

name                0
description        56
make                0
model               0
type                0
year                0
price              23
engine              2
cylinders         105
fuel                7
mileage            34
transmission        2
trim                1
body                3
doors               7
exterior_color      5
interior_color     38
drivetrain          0
dtype: int64


In [49]:
df_cleaned = df.dropna(subset=['mileage', 'price'])
df = df_cleaned

#### Fitting a model

In [50]:
# creating predictor and response variables
X = df[['make', 'year', 'mileage']]
y = df['price']

In [51]:
# convert categoricals into 1-0 indicators
X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


In [52]:
X_train.head()

,year,mileage,make_Audi,make_BMW,make_Buick,make_Cadillac,make_Chevrolet,make_Chrysler,make_Dodge,make_Ford,...,make_Lexus,make_Lincoln,make_Mazda,make_Mercedes-Benz,make_Nissan,make_RAM,make_Subaru,make_Toyota,make_Volkswagen,make_Volvo
148,2024,11.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
597,2024,15.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
317,2024,2.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
920,2024,7.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
890,2024,0.0,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [54]:

# Save the trained model
joblib.dump(model, 'linear_model.pkl')

['linear_model.pkl']

In [58]:
!pwd
!ls

/content/ci-cd-project
data_analysis  linear_model.pkl  README.md  requirements.txt  tests


### Commit to Gibhub

In [55]:
#%cd /content/ci-cd-project/data_analysis
#!pwd
#!ls
#!ls /content/drive/'My Drive'/'Colab Notebooks'
!ls /content/ci-cd-project/data_analysis/

import_data.py	Kaggle_Cars_2024_Data_Summary.ipynb


In [56]:
# copy this file into the Github repo
!cp '/content/drive/My Drive/Colab Notebooks/Kaggle_Cars_2024_Data_Summary.ipynb' /content/ci-cd-project/data_analysis/

In [21]:

#!ll /content/ci-cd-project/data_analysis/

In [21]:
# make the commit of just this notebook
!git add data_analysis/Kaggle_Cars_2024_Data_Summary.ipynb
!git commit -m "add Kaggle data to  .ipynb notebook"
!git push origin main

[main ee31f31] add Kaggle data to  .ipynb notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite data_analysis/Kaggle_Cars_2024_Data_Summary.ipynb (95%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 9.16 KiB | 4.58 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/empt22/ci-cd-project.git
   1a96e8b..ee31f31  main -> main


In [ ]:
# end of file